In [1]:
# Test the structure of the result object
'''
qc_test = create_qcbm_circuit(8, np.random.uniform(0, 2*np.pi, 16))
qc_test.measure_all()
result_test = sampler.run([qc_test]).result()
pub_result = result_test[0]
bit_array = pub_result.data.meas
counts = dict(bit_array.get_counts())
print("Counts sample:")
for k, v in list(counts.items())[:5]:
    print(f"  Key: {k} (type: {type(k)}), Value: {v}")
print(f"Total keys: {len(counts)}")
print(f"Total shots: {sum(counts.values())}")
'''

'\nqc_test = create_qcbm_circuit(8, np.random.uniform(0, 2*np.pi, 16))\nqc_test.measure_all()\nresult_test = sampler.run([qc_test]).result()\npub_result = result_test[0]\nbit_array = pub_result.data.meas\ncounts = dict(bit_array.get_counts())\nprint("Counts sample:")\nfor k, v in list(counts.items())[:5]:\n    print(f"  Key: {k} (type: {type(k)}), Value: {v}")\nprint(f"Total keys: {len(counts)}")\nprint(f"Total shots: {sum(counts.values())}")\n'

In [2]:
import pandas as pd

# Load your dataset
df = pd.read_csv('syntheticQDstructure.csv')
print(df.head())
print("Shape:", df.shape)

  core_composition ligand_composition  size (nm)  band_gap (eV)  \
0             InAs               TDPA        3.6           0.82   
1             ZnSe      Zinc stearate        3.8           3.07   
2             PbSe                MPA        6.9           1.06   
3              ZnS     Hexadecylamine        2.3           3.98   
4              InP                TOP        6.3           1.94   

   quantum_yield (%)  emission_maximum (nm) lattice_type  a (Å)  b (Å)  c (Å)  \
0                 64                    904  Zinc Blende   6.06   6.06   6.06   
1                 71                    544  Zinc Blende   5.67   5.67   5.67   
2                 32                    905    Rock Salt   6.12   6.12   6.12   
3                 45                    442        Cubic   5.41   5.41   5.41   
4                 70                    634  Zinc Blende   5.87   5.87   5.87   

   alpha (°)  beta (°)  gamma (°)  volume (Å³)  
0       90.0      90.0       90.0       222.55  
1       90.0

In [3]:
df_encoded = pd.get_dummies(df)
print(df_encoded.head())
print("Encoded Shape:", df_encoded.shape)

   size (nm)  band_gap (eV)  quantum_yield (%)  emission_maximum (nm)  a (Å)  \
0        3.6           0.82                 64                    904   6.06   
1        3.8           3.07                 71                    544   5.67   
2        6.9           1.06                 32                    905   6.12   
3        2.3           3.98                 45                    442   5.41   
4        6.3           1.94                 70                    634   5.87   

   b (Å)  c (Å)  alpha (°)  beta (°)  gamma (°)  ...  ligand_composition_OLA  \
0   6.06   6.06       90.0      90.0       90.0  ...                   False   
1   5.67   5.67       90.0      90.0       90.0  ...                   False   
2   6.12   6.12       90.0      90.0       90.0  ...                   False   
3   5.41   5.41       90.0      90.0       90.0  ...                   False   
4   5.87   5.87       90.0      90.0       90.0  ...                   False   

   ligand_composition_Oleic acid  liga

In [4]:
import torch

X_array = df_encoded.to_numpy(dtype='float32')
X_tensor = torch.tensor(X_array)

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

dataset = TensorDataset(X_tensor)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Autoencoder architecture
class Autoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super().__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 16),
            nn.ReLU(),
            nn.Linear(16, input_dim)
        )

    def forward(self, x):
        z = self.encoder(x)
        x_recon = self.decoder(z)
        return x_recon

# Define model, loss, optimizer
input_dim = X_tensor.shape[1]
latent_dim = 2  # small latent space for quantum training
model = Autoencoder(input_dim, latent_dim)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train autoencoder
for epoch in range(50):
    total_loss = 0
    for batch in loader:
        optimizer.zero_grad()
        recon = model(batch[0])
        loss = criterion(recon, batch[0])
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

# Save encoded data
with torch.no_grad():
    encoded_data = model.encoder(X_tensor).numpy()
    
print("Encoded Data Shape:", encoded_data.shape)
print("Encoded Data Sample:", encoded_data[:5])

Epoch 1, Loss: 61518.1465
Epoch 2, Loss: 55954.9141
Epoch 3, Loss: 34796.3447
Epoch 4, Loss: 13436.8011
Epoch 5, Loss: 6141.8463
Epoch 6, Loss: 3758.0370
Epoch 7, Loss: 3214.3361
Epoch 8, Loss: 3405.9587
Epoch 9, Loss: 1640.1172
Epoch 10, Loss: 1221.8106
Epoch 11, Loss: 1281.4724
Epoch 12, Loss: 997.4612
Epoch 13, Loss: 861.8401
Epoch 14, Loss: 749.1600
Epoch 15, Loss: 751.9536
Epoch 16, Loss: 738.7723
Epoch 17, Loss: 717.8904
Epoch 18, Loss: 692.0080
Epoch 19, Loss: 633.4734
Epoch 20, Loss: 583.0721
Epoch 21, Loss: 630.5296
Epoch 22, Loss: 598.0688
Epoch 23, Loss: 620.1712
Epoch 24, Loss: 604.3057
Epoch 25, Loss: 633.6089
Epoch 26, Loss: 550.0746
Epoch 27, Loss: 526.4237
Epoch 28, Loss: 518.5535
Epoch 29, Loss: 526.1904
Epoch 30, Loss: 577.2879
Epoch 31, Loss: 526.5234
Epoch 32, Loss: 565.1510
Epoch 33, Loss: 578.0210
Epoch 34, Loss: 527.5343
Epoch 35, Loss: 558.4906
Epoch 36, Loss: 587.6406
Epoch 37, Loss: 539.9879
Epoch 38, Loss: 543.2601
Epoch 39, Loss: 519.0472
Epoch 40, Loss: 518

In [6]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
encoded_normalized = scaler.fit_transform(encoded_data)  # shape: (N, 2)


In [7]:
def float_to_bits(x, n_bits=4):
    """Convert a float in [0,1] to fixed-length binary string"""
    max_int = 2**n_bits - 1
    int_val = int(x * max_int)
    return format(int_val, f'0{n_bits}b')

n_bits = 4  # bits per latent dimension
bitstrings = []

for point in encoded_normalized:
    bits = ''.join(float_to_bits(x, n_bits) for x in point)
    bitstrings.append(bits)

print("Bitstrings Sample:", bitstrings[:5])

Bitstrings Sample: ['00100010', '10111100', '00100010', '11101110', '10011001']


In [8]:
from qiskit import QuantumCircuit
import numpy as np

def create_qcbm_circuit(num_qubits, parameters):
    qc = QuantumCircuit(num_qubits)
    idx = 0
    for i in range(num_qubits):
        qc.ry(parameters[idx], i)
        idx += 1
        qc.rz(parameters[idx], i)
        idx += 1
    for i in range(num_qubits - 1):
        qc.cx(i, i + 1)
    return qc


In [9]:
from qiskit.primitives import StatevectorSampler
from scipy.optimize import minimize
from collections import Counter

sampler = StatevectorSampler()

def compute_kl_divergence(p_data, p_model, epsilon=1e-8):
    # Ensure support match
    keys = set(p_data.keys()).union(set(p_model.keys()))
    return sum(
        p_data.get(k, 0) * np.log((p_data.get(k, epsilon) + epsilon) / (p_model.get(k, epsilon) + epsilon))
        for k in keys
    )

def evaluate_qcbm_loss(parameters, num_qubits, target_distribution):
    qc = create_qcbm_circuit(num_qubits, parameters)
    # Add measurements
    qc.measure_all()
    result = sampler.run([qc]).result()
    # In Qiskit 2.x, the result format is different
    pub_result = result[0]  # Get the first (and only) pub result
    bit_array = pub_result.data.meas
    counts = dict(bit_array.get_counts())
    # Normalize output - keys are already strings
    total_shots = sum(counts.values())
    model_distribution = {k: v / total_shots for k, v in counts.items()}
    return compute_kl_divergence(target_distribution, model_distribution)

def compute_data_distribution(binary_encoded_data):
    str_data = [''.join(str(bit) for bit in sample) for sample in binary_encoded_data]
    counts = Counter(str_data)
    total = sum(counts.values())
    return {k: v / total for k, v in counts.items()}

In [10]:
# Convert bitstrings to binary array format
binary_encoded_data = np.array([[int(bit) for bit in bitstring] for bitstring in bitstrings])

num_qubits = binary_encoded_data.shape[1]
num_parameters = 2 * num_qubits
initial_params = np.random.uniform(0, 2 * np.pi, num_parameters)

target_distribution = compute_data_distribution(binary_encoded_data)

result = minimize(
    evaluate_qcbm_loss,
    x0=initial_params,
    args=(num_qubits, target_distribution),
    method='COBYLA',
    options={'maxiter': 100}
)

trained_params = result.x
print(f"Optimization completed. Final loss: {result.fun}")
print(f"Number of qubits: {num_qubits}")
print(f"Target distribution (first 5 items): {dict(list(target_distribution.items())[:5])}")

Optimization completed. Final loss: 5.338440470229853
Number of qubits: 8
Target distribution (first 5 items): {'00100010': 0.14, '10111100': 0.01, '11101110': 0.11, '10011001': 0.05, '11011100': 0.04}


In [11]:
qc_trained = create_qcbm_circuit(num_qubits, trained_params)
qc_trained.measure_all()
result = sampler.run([qc_trained], shots=1000).result()

# Get the counts and convert to distribution
pub_result = result[0]
bit_array = pub_result.data.meas
counts = dict(bit_array.get_counts())
total_shots = sum(counts.values())
samples = {k: v / total_shots for k, v in counts.items()}

# Sort samples by probability
sorted_samples = sorted(samples.items(), key=lambda x: x[1], reverse=True)
top_samples = [k for k, _ in sorted_samples[:10]]
decoded_inputs = [np.array([int(bit) for bit in s]) for s in top_samples]

print("Top 10 most probable samples:")
for i, (sample, prob) in enumerate(sorted_samples[:10]):
    print(f"{i+1}. {sample}: {prob:.4f}")

Top 10 most probable samples:
1. 01001010: 0.0620
2. 10101010: 0.0550
3. 10110010: 0.0510
4. 01011010: 0.0450
5. 10100010: 0.0410
6. 01000010: 0.0370
7. 10111010: 0.0360
8. 01010010: 0.0330
9. 11001010: 0.0300
10. 01010100: 0.0290


In [12]:
# Convert bit strings back to 2D latent space format
def bits_to_float(bits, n_bits=4):
    """Convert binary string to float in [0,1]"""
    max_int = 2**n_bits - 1
    int_val = int(bits, 2)
    return int_val / max_int

# Convert the top samples back to 2D latent vectors
latent_vectors = []
for sample in top_samples:
    # Split the 8-bit string into two 4-bit parts
    bits_dim1 = sample[:4]  # First 4 bits
    bits_dim2 = sample[4:]  # Last 4 bits
    
    # Convert back to floats in [0,1]
    float_dim1 = bits_to_float(bits_dim1, n_bits)
    float_dim2 = bits_to_float(bits_dim2, n_bits)
    
    latent_vectors.append([float_dim1, float_dim2])

# Convert to tensor with proper scaling (inverse of normalization)
latent_tensor = torch.tensor(latent_vectors, dtype=torch.float32)
# Scale back to original latent space range using the fitted scaler
latent_scaled = scaler.inverse_transform(latent_tensor.numpy())
latent_scaled_tensor = torch.tensor(latent_scaled, dtype=torch.float32)

model.decoder.eval()  # Set to eval mode (no dropout/batchnorm)

with torch.no_grad():
    reconstructed_outputs = model.decoder(latent_scaled_tensor)
    
synthetic_results = reconstructed_outputs.numpy()
print("Synthetic results shape:", synthetic_results.shape)
print("Synthetic results sample:", synthetic_results[:3])
print("Latent vectors shape:", latent_scaled_tensor.shape)
print("Latent vectors sample:", latent_scaled_tensor[:3])

Synthetic results shape: (10, 34)
Synthetic results sample: [[ 1.29270542e+00 -7.00705230e-01  6.47788696e+01  7.61845093e+02
   7.79755783e+00 -2.72967982e+00 -1.18559372e+00  1.05435463e+02
   9.10030060e+01  9.95890350e+01  2.15124298e+02 -2.14780375e-01
  -7.76464319e+00 -8.91425133e+00 -1.23014545e+00 -1.41232061e+01
  -1.05889091e+01 -1.20804894e+00 -7.34146118e-01  7.86216497e+00
   1.04454031e+01 -1.02548437e+01 -3.01259637e+00 -1.13429985e+01
  -8.28905964e+00 -2.06592226e+00 -2.17754936e+00  7.17689085e+00
  -1.30619125e+01 -8.39052486e+00  1.21902103e+01  5.36464071e+00
   1.37099111e+00  2.17105923e+01]
 [ 2.06064343e+00  9.01140690e-01  4.58947754e+01  6.35134399e+02
   3.91183090e+00  3.96530056e+00  3.60622859e+00  7.98957825e+01
   7.86123199e+01  8.30608978e+01  1.70213974e+02  1.13342226e-01
   1.26197860e-02 -1.27786398e-01  2.27783918e-02  7.89934397e-01
  -3.53521168e-01 -2.81317905e-02  1.01453662e-01  1.01163960e+00
  -2.53309846e-01 -1.04592597e+00 -1.33004487e-

In [13]:

categorical_cols = ['core_composition', 'ligand_composition', 'lattice_type']

dummy_columns = [col for col in df_encoded.columns if any(col.startswith(cat + '_') for cat in categorical_cols)]
synthetic_df = pd.DataFrame(synthetic_results, columns=df_encoded.columns)
for col in categorical_cols:
    # Extract relevant dummy columns
    dummy_subset = synthetic_df[[c for c in synthetic_df.columns if c.startswith(col + '_')]]
    
    # Find argmax (i.e., the "1") to get back the original category
    synthetic_df[col] = dummy_subset.idxmax(axis=1).str.replace(col + '_', '')

    # Drop the dummy columns
    synthetic_df.drop(columns=dummy_subset.columns, inplace=True)
    
column_order = df.columns.tolist()
synthetic_df = synthetic_df.reindex(columns=column_order)

print("Synthetic DataFrame head:")
print(synthetic_df.head())

Synthetic DataFrame head:
  core_composition ligand_composition  size (nm)  band_gap (eV)  \
0             ZnSe                DDA   1.292705      -0.700705   
1             ZnSe                 OA   2.060643       0.901141   
2              InP      Zinc stearate   4.650207       9.247084   
3             ZnSe                DDA   1.363221      -1.201594   
4              InP      Zinc stearate   4.479449       8.408709   

   quantum_yield (%)  emission_maximum (nm) lattice_type     a (Å)      b (Å)  \
0          64.778870             761.845093  Zinc Blende  7.797558  -2.729680   
1          45.894775             635.134399  Zinc Blende  3.911831   3.965301   
2          39.282303             726.138855    Hexagonal -5.082218  14.182417   
3          61.308781             740.915161  Zinc Blende  7.803060  -1.310134   
4          42.189487             747.397583    Hexagonal -3.948589  13.292591   

      c (Å)   alpha (°)   beta (°)  gamma (°)  volume (Å³)  
0 -1.185594  105.435463

In [14]:
synthetic_df.to_csv("synthetic_results.csv", index=False)